In [ ]:
import boto3
import pprint
from boto3.dynamodb.conditions import Key

In [ ]:
print (dataset_table)

In [ ]:
region_name = 'us-east-1'
dynamodb = boto3.resource('dynamodb', region_name=region_name)

In [58]:
def get_table_stuff(table1):
    
    response = table1.get_item(Key={'dataset_id': 'henlo'})
    
    return response['Item']

abc = get_table_stuff(table)
print (abc)

{'app': 'asfpsfafas', 'num_devices': Decimal('600'), 'attributeTypes': ['N', 'N'], 'attributeRangeMaxes': ['1000', '12'], 'logo_image_url': 'bingus', 'attributeRangeMins': ['600', '4'], 'placeholder': 'placeholder', 'category': 'Consumer', 'dataset_id': 'henlo', 'attributes': ['wapapa', 'wowowow'], 'name': 'bingus'}


#### Simply building a fake dataset off of one example

In [59]:
# takes in a dataset id, converts it into a dictionary

# never making fake data from another table, so no point not just declaring it
table = dynamodb.Table('dataset_table')

def preprocess_dynamo_row(column_header, input):

    response = table.get_item(Key={column_header: input})
    return response['Item']

print (preprocess_dynamo_row('dataset_id', 'henlo'))

{'app': 'asfpsfafas', 'num_devices': Decimal('600'), 'attributeTypes': ['N', 'N'], 'attributeRangeMaxes': ['1000', '12'], 'logo_image_url': 'bingus', 'attributeRangeMins': ['600', '4'], 'placeholder': 'placeholder', 'category': 'Consumer', 'dataset_id': 'henlo', 'attributes': ['wapapa', 'wowowow'], 'name': 'bingus'}


In [62]:
# this function will grab the relevant rows (column header, min, max, num_rows)
def grab_relevant_info(column_header, input):

    dict = preprocess_dynamo_row(column_header, input)

    columns = []
    schema = []
    num_rows = 0
    ranges = []

    # grab the column headers
    columns = dict['attributes']

    # grab the data schema
    schema = dict['attributeTypes']

    # grab the number of rows (fake data)
    num_rows = int(dict['num_devices']) # this avoids any problem with that it is a Decimal value

    # create the ranges
    mins = dict['attributeRangeMins']
    maxs = dict['attributeRangeMaxes']

    # corresponding min and max of column
    for i in range(len(mins)):
        ranges.append([])
        ranges[i].append(float(mins[i]))
        ranges[i].append(float(maxs[i]))

    return columns, schema, num_rows, ranges

test_data = grab_relevant_info('dataset_id', 'henlo')
print (test_data)
test_columns = test_data[0]
test_schema = test_data[1]
test_num_rows = test_data[2]
test_ranges = test_data[3]

(['wapapa', 'wowowow'], ['N', 'N'], 600, [[600.0, 1000.0], [4.0, 12.0]])


#### The below function takes in column, range, and the number of rows and produces randomized integers in the range of each column

In [64]:

import random
# inputs: columns, discrete/continuous, ranges for each column, number of rows
def fakedata(columns, range, num_rows):
    
    # input validation
    if not (len(columns) == len(range)):
        return ("incorrect lengths of columns in one of the first three inputs")
    elif (num_rows <= 0):
        return ("incorrect number of rows")
    
    output_data = [columns]

    col_iterator = 0
    row_of_data = 1 # topline is the headers
    # write an array where each item is a row of fake data
    while row_of_data <= num_rows:
        output_data.append([])
        while col_iterator < len(columns):
            output_data[row_of_data].append(random.randint(range[col_iterator][0], range[col_iterator][1]))
            col_iterator += 1
        
        col_iterator = 0
        row_of_data += 1
        
    return output_data

# this takes in the input created from
fakedata(test_columns, test_ranges, test_num_rows)

[['wapapa', 'wowowow'],
 [988, 11],
 [724, 5],
 [865, 8],
 [765, 12],
 [926, 10],
 [891, 11],
 [864, 12],
 [858, 11],
 [834, 5],
 [975, 12],
 [612, 9],
 [836, 7],
 [766, 11],
 [759, 10],
 [969, 5],
 [910, 8],
 [940, 8],
 [715, 5],
 [908, 12],
 [614, 9],
 [723, 9],
 [624, 5],
 [937, 8],
 [698, 4],
 [680, 11],
 [872, 10],
 [800, 8],
 [727, 8],
 [842, 4],
 [741, 10],
 [1000, 7],
 [634, 7],
 [852, 9],
 [889, 8],
 [794, 9],
 [954, 8],
 [987, 7],
 [985, 6],
 [914, 8],
 [737, 12],
 [895, 4],
 [661, 6],
 [875, 7],
 [680, 9],
 [981, 10],
 [852, 6],
 [616, 5],
 [630, 12],
 [695, 6],
 [666, 11],
 [607, 12],
 [908, 7],
 [750, 9],
 [736, 7],
 [793, 4],
 [875, 9],
 [784, 6],
 [927, 7],
 [794, 10],
 [701, 6],
 [665, 11],
 [796, 8],
 [684, 9],
 [712, 8],
 [988, 11],
 [917, 5],
 [802, 6],
 [863, 7],
 [924, 12],
 [739, 5],
 [662, 12],
 [883, 10],
 [616, 6],
 [650, 11],
 [699, 10],
 [922, 10],
 [913, 10],
 [834, 12],
 [830, 11],
 [835, 5],
 [804, 9],
 [692, 9],
 [802, 9],
 [976, 4],
 [619, 5],
 [771, 8],

In [66]:
# inputs: columns, discrete/continuous, ranges for each column, correlation, number of rows
# this has every column besides the first column be a function of the first column
# correlation: index of length n, 0 means no correlation with anything else, any non-zero int needs to have a pair (should be validated, will not be yet)
# assume earlier index is the one the later one is based on... doesn't really matter
# NOTE - AS OF NOW CAN ONLY TAKE IN 'D', 'B', or 'N'
def schematic_fake_data(columns, schema, ranges, num_rows):
    
    # input validation
    if not (len(columns) == len(ranges)):
        return ("incorrect lengths of columns in one of the first three inputs")
    elif (num_rows <= 0):
        return ("incorrect number of rows")

    # preprocessing, if binary, turn range into 0, 1, schema into N
    for i in range(len(schema)):
        if schema[i] == 'B':
            schema[i] = 'N'
            ranges[i] = [0, 1]

    # # this is a dictionary on that tracks the counts of each value in array
    # correlation_counter_dict = {}
    # validation_counter = 0

    # while validation_counter < len(correlation):
    #     if correlation[validation_counter] in correlation_counter_dict.keys():
    #         correlation_counter_dict[correlation[validation_counter]] += 1
    #     else:
    #         correlation_counter_dict[correlation[validation_counter]] = 1

    #     validation_counter += 1
    # val_list = (list(correlation_counter_dict.values()))
    # key_list = (list(correlation_counter_dict.keys()))
    # for i in val_list:
    #     if i < 2:
    #         if key_list[val_list.index(i)] != 0:
    #             return ("error: need a corresponding value with this correlation value")
    
    output_data = [columns]

    col_iterator = 0
    row_of_data = 1 # topline is the headers
    # write an array where each item is a row of fake data

    while row_of_data <= num_rows:

        output_data.append([])

        while (len(columns) > col_iterator):

            # # if this is a "correlated column", need to find other column and make it a function of another  
            # if (correlation[col_iterator] != 0):
                
            #     # this finds if this current column is later
            #     temp = 0
            #     while temp < col_iterator:

            #         # this means that this column is later
            #         if correlation[temp] == correlation[col_iterator]:
            #             output_data[row_of_data].append(output_data[row_of_data][temp] * 3 + random.random() * 5)
            #             temp = col_iterator + 2 # to cut out of the loop
            #             col_iterator += 1
            #         else: 
            #             temp +=1
                
            #     # if this column index is earlier, make it random
            #     if (schema[col_iterator] == "int"):
            #         output_data[row_of_data].append(random.randint(range[col_iterator][0], range[col_iterator][1]))
            #         col_iterator += 1
            #     else:
            #         output_data[row_of_data].append(range[col_iterator][0] + random.random() * (range[col_iterator][1] - range[col_iterator][0]))
            #         col_iterator += 1

            #else: 
            # if an integer, fake data
            if (schema[col_iterator] == "N"):
                output_data[row_of_data].append(random.randint(ranges[col_iterator][0], ranges[col_iterator][1]))
                col_iterator += 1

            # if not an int, a double
            else:
                output_data[row_of_data].append(ranges[col_iterator][0] + random.random() * (ranges[col_iterator][1] - ranges[col_iterator][0]))
                col_iterator += 1

        col_iterator = 0
        row_of_data += 1
        
    return output_data


schematic_fake_data(test_columns, ['D', 'B'], test_ranges, test_num_rows)

[['wapapa', 'wowowow'],
 [832.8189396956657, 0],
 [746.1665961629019, 1],
 [649.9280198556493, 1],
 [877.585836767612, 1],
 [659.3746660687135, 1],
 [740.3348574360346, 0],
 [932.3832421149561, 1],
 [770.6287050839148, 1],
 [872.9461057668764, 1],
 [897.7624793712616, 1],
 [900.2394256652076, 0],
 [610.9772618264902, 1],
 [949.9953552236443, 0],
 [610.8113541776708, 1],
 [748.2303180810708, 1],
 [975.83869349608, 0],
 [840.4587130297602, 0],
 [928.2021371298331, 0],
 [713.2333644672532, 1],
 [736.2618459280503, 0],
 [664.0219471044911, 1],
 [923.763543967395, 1],
 [886.8848277950553, 1],
 [842.638142988384, 1],
 [991.4209639332155, 0],
 [687.6646351466685, 0],
 [753.978789306539, 1],
 [736.259914305508, 1],
 [763.4519519777572, 0],
 [906.1773604210948, 0],
 [974.715877482332, 0],
 [761.0690746070152, 0],
 [714.6146080766263, 0],
 [725.095913756704, 0],
 [874.1153722614615, 0],
 [827.8805277407494, 1],
 [695.7061809520599, 1],
 [692.1221828124133, 1],
 [860.7290799436078, 1],
 [984.0224

In [69]:
import csv

def convert_to_csv(columns, schema, ranges, num_rows, file_name):
    output = schematic_fake_data(columns, schema, ranges, num_rows)

    with open(file_name, 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        x = 0
        while x < len(output):
            spamwriter.writerow(output[x])
            x += 1

convert_to_csv(test_columns, test_schema, test_ranges, test_num_rows, 'fakedata/no_correlation.csv')

In [73]:
def query_to_csv(column_header, input):
    # first, get the dictionary
    right_info = grab_relevant_info(column_header, input)

    print (right_info)
    # parse the right info into the four values we need
    test_columns = right_info[0]
    test_schema = right_info[1]
    test_num_rows = right_info[2]
    test_ranges = right_info[3]
    
    # write the csv
    filename = ('fakedata/' + input + '.csv')
    convert_to_csv(test_columns, test_schema, test_ranges, test_num_rows, filename)

query_to_csv('dataset_id', 'henlo')

(['wapapa', 'wowowow'], ['N', 'N'], 600, [[600.0, 1000.0], [4.0, 12.0]])
